# Abel & Josette's AI Term Project
This is our workspace for collaborative filtering and content based music generation.
For spotify API, reference this page for [setup](https://medium.com/@benfenison/using-spotify-web-api-w-python-6b31a328b26e) and this page for [documentation](https://github.com/plamere/spotipy). Data for the content base filtering was collected [here](https://grinslade.pythonanywhere.com).

## The Problem
Our project was focused on the pratical everyday issue that music listeners struggle to find new songs that they would *actually* listen to. There are many manys to solve this problem, but the most common way that is used in this industry is collaborative filtering. Collaboraive filtering has been done by Spotify, Netflix and many others for recommendations to their users. Collaborative filtering is an algorithm that dictates what users are similar and then suggest content that the similar users enjoy. Another method commonly used in pratice is Content-based filtering. This algorithm simply decides if a user's liked content is similar to other content based off its metadata or attributes then suggests this content. 

## Our Approach
Our approach to this problem similar but with our own implementation. More specifically, we would create two ways of generating suggested songs: Content-based filtering and Collaborative filtering. We would implement these algorithms in our own way and compare it to established generations like Spotify's. We decided we would look at user data to generate 5 songs that the listener would truly enjoy.

## Results

## If we had more time

If we had more time for this project, we would have perfected our algorithms and get access from spotify to use a bigger data set instead of .

In [13]:
import sys
import spotipy
import spotipy.util as util

sc = 'user-top-read'

username = input("Enter username or none: ")
results = {}
if username == "none":
    username = "jojogrin"
token = util.prompt_for_user_token(username=username,scope=sc,client_id='33ebd71536d94caebcf54970eaf4191a', client_secret='87c219c0663d414388f96a6b7da7ee11', redirect_uri='https://grinslade.pythonanywhere.com/api_callback')
results = sp.current_user_top_tracks()

Enter username or none: none


In [14]:
sp = spotipy.Spotify(auth=token)

# User's Top Tracks

In [15]:
import pandas as pd 

user_data = {}
songs = []
ids = []
count = 0
print(username + "'s Top 20 songs")
print("----------------------------------")
for track in results['items']:
    count += 1
    songs.append(track)
    ids.append(track['id'])
    print(str(count) + '. ' + track['name'] + ' - ' + track['artists'][0]['name'])
    
features = sp.audio_features(ids)

for item in features:
    item.update( {"user": username})
user = pd.DataFrame(features)


jojogrin's Top 20 songs
----------------------------------
1. Every Single Thing - HOMESHAKE
2. Look What You Made Me Do - Taylor Swift
3. thank u, next - Ariana Grande
4. How Long - Charlie Puth
5. God is a woman - Ariana Grande
6. Bonkers - Dizzee Rascal
7. Confident - Demi Lovato
8. Truth Hurts - Lizzo
9. 7 rings - Ariana Grande
10. Can't Fight The Moonlight - LeAnn Rimes
11. J'veux sortir avec un rappeur - Alice et Moi
12. Fitness - Lizzo
13. Bitch Better Have My Money - Rihanna
14. 6 Inch (feat. The Weeknd) - Beyoncé
15. G.O.M.D. - J. Cole
16. Burn It Up - HIGH HØØPS
17. Can't Nobody Love You - Solomon Burke
18. I Got It - Manus
19. SICKO MODE - Skrillex Remix - Travis Scott
20. POWER - Kanye West


# Spotify Suggestions
Results are different every request even with same seed

In [16]:
print('Spotify Suggestions from %s\'s Top Song %s - %s' % (username, songs[0]['name'], songs[0]['artists'][0]['name']))
print("----------------------------------")
spot_results = sp.recommendations(seed_tracks=[ids[0]])
for track in spot_results['tracks']:
    print(track['name'] + ' - ' + track['artists'][0]['name'])

Spotify Suggestions from jojogrin's Top Song Every Single Thing - HOMESHAKE
----------------------------------
Bones - Crumb
Way to be Loved - TOPS
Blue Train Lines - Mount Kimbie
Seattle Party - Chastity Belt
I'm The Man, That Will Find You - Connan Mockasin
Dope on a Rope - The Growlers
Out Getting Ribs - King Krule
Suddenly - Drugdealer
Windowpane - Mild High Club
23 - Rejjie Snow
Lucy - Still Woozy
The Way Things Change - Yellow Days
Teenage Blue - Dreamgirl
Mirage - Toro y Moi
Another Weekend - Ariel Pink
Friends - levitation room
When I Get Home - Post Animal
Rare Hearts - The Growlers
Hunnybee - Unknown Mortal Orchestra
Easy Easy - King Krule


# Content-based Suggestions
Standard Deviation of danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence and tempo

In [5]:
import time
t0 = time.time()

#user.to_csv(username + "_top.csv", index=True)
data = pd.read_csv('mock_spot_data.csv')
scores = []
song_index = []
possible = 11
norm_prob = []

for topindex, toprow in data.iterrows():
    score = 0
    if not(data.iloc[topindex]['user'] == username):
        for index, row in user.iterrows():
            if abs(user.iloc[index]['danceability'] - data.iloc[topindex]['danceability']) <= user['danceability'].std():
                score+=1
            if abs(user.iloc[index]['energy'] - data.iloc[topindex]['energy']) <= user['energy'].std():
                score+=1
            if abs(user.iloc[index]['key'] - data.iloc[topindex]['key']) <= user['key'].std():
                score+=1
            if abs(user.iloc[index]['loudness'] - data.iloc[topindex]['loudness']) <= user['loudness'].std():
                score+=1
            if abs(user.iloc[index]['mode'] - data.iloc[topindex]['mode']) <= user['mode'].std():
                score+=1
            if abs(user.iloc[index]['speechiness'] - data.iloc[topindex]['speechiness']) <= user['speechiness'].std():
                score+=1
            if abs(user.iloc[index]['acousticness'] - data.iloc[topindex]['acousticness']) <= user['acousticness'].std():
                score+=1
            if abs(user.iloc[index]['instrumentalness'] - data.iloc[topindex]['instrumentalness']) <= user['instrumentalness'].std():
                score+=1
            if abs(user.iloc[index]['liveness'] - data.iloc[topindex]['liveness']) <= user['liveness'].std():
                score+=1
            if abs(user.iloc[index]['valence'] - data.iloc[topindex]['valence']) <= user['valence'].std():
                score+=1
            if abs(user.iloc[index]['tempo'] - data.iloc[topindex]['tempo']) <= user['tempo'].std():
                 score+=1
        scores.append(score/11)
        song_index.append(topindex)
    #print("Score: ", score, " out of ", possible)
    
#normalize the scores and get prob 
min_x = min(scores)
max_x = max(scores)

suggested_songs = []

def sortSecond(val): 
    return val[1]

for i in range(len(scores)):
    x = (scores[i] - min_x) / (max_x - min_x)
    x = min((x),1)
    norm_prob.append(x)
    index = song_index[i]
    #print('%s - %s : Probability - %.2f' % (sp.track(data.iloc[index]['uri'])['name'], sp.track(data.iloc[index]['uri'])['artists'][0]['name'], norm_prob[i]))
    if x >= 0.70: # this threshold can be changed!
        suggested_songs.append((index, x))
        
suggested_songs.sort(key = sortSecond)
top_songs_n = len(suggested_songs) - 6
if 5 > len(suggested_songs):
    top_songs_n = 0
    
print('Content-based Suggestions')
print("----------------------------------")
for n in range(len(suggested_songs)-1, top_songs_n, -1):
    song_i = suggested_songs[n][0]
    prob = suggested_songs[n][1]
    print(sp.track(data.iloc[song_i]['uri'])['name'], '-', sp.track(data.iloc[song_i]['uri'])['artists'][0]['name'])
total = time.time() - t0
print("Runtime:", total)

Content-based Suggestions
----------------------------------
Freaking Out - A R I Z O N A
Strings - iann dior
Truth Hurts - Lizzo
All Love - FLETCHER
Not Afraid Anymore - Halsey
Runtime: 102.78352379798889


# Collaborative Filter